In [1]:
import sys
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import warnings
from astropy.convolution import Gaussian1DKernel
from astropy.convolution import convolve
from scipy.signal import find_peaks
from scipy.optimize import curve_fit



# import lightkurve as lk
# from scipy.optimize import curve_fit

# from astropy.convolution import Gaussian1DKernel
# from astropy.convolution import convolve
sys.path.insert(0, "../")

from michael import janet

Hi there. Welcome to the good place.


In [2]:
uni = pd.read_csv("/Users/oliver hall/Research/unicorn/data/unicorn.csv", index_col=0)

In [6]:
uni.columns

Index(['dr2_source_id', 'angular_distance', 'source_id', 'r_est',
       'r_lo_photogeo', 'r_hi_photogeo', 'ra', 'ra_error', 'dec', 'dec_error',
       'parallax', 'parallax_error', 'astrometric_gof_al',
       'astrometric_params_solved', 'nu_eff_used_in_astrometry',
       'pseudocolour', 'pseudocolour_error', 'visibility_periods_used', 'ruwe',
       'phot_g_mean_flux', 'phot_g_mean_flux_error', 'phot_g_mean_mag',
       'phot_bp_mean_flux', 'phot_bp_mean_flux_error', 'phot_bp_mean_mag',
       'phot_rp_mean_flux', 'phot_rp_mean_flux_error', 'phot_rp_mean_mag',
       'bp_rp', 'ecl_lat', 'u', 'v', 'w', 'sep', 'Gmag', 'e_Gmag',
       'e_phot_bp_mean_mag', 'e_phot_rp_mean_mag', 'e_bp_rp', 'MGna',
       'binflag', 'gal_teff', 'gal_e_teff', 'gal_logg', 'gal_e_logg',
       'gal_fe_h', 'gal_e_fe_h', 'gal_alpha_fe', 'gal_e_alpha_fe', 'apo_teff',
       'apo_e_teff', 'apo_logg', 'apo_e_logg', 'apo_m_h', 'apo_e_m_h',
       'apo_fe_h', 'apo_e_fe_h', 'lam_teff', 'lam_e_teff', 'lam_logg',
 

In [3]:
j = janet.boot(uni, index=0)


ValueError: When using the prior function you must provide observables as input.

In [ ]:
j.run(period_range = (1., 13.))